# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
va_city_data_df = pd.read_csv("../WeatherPy/city_data.csv")
va_city_data_df.head()

,Unnamed: 0,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mount Isa,-20.73,139.50,68.00,56,0,8.05,AU,1603572799
1,1,Norman Wells,65.28,-126.83,21.20,92,75,4.70,CA,1603572799
2,2,Ushuaia,-54.80,-68.30,50.00,70,75,8.05,AR,1603572717
3,3,Albany,42.60,-73.97,55.99,68,45,14.00,US,1603572678
4,4,Thompson,55.74,-97.86,24.80,53,90,9.17,CA,1603572586


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
locations = va_city_data_df[["Lat", "Lng"]].astype(float)
locations.dtypes

Lat    float64
Lng    float64
dtype: object

In [5]:
humidity = va_city_data_df["Humidity"]
humidity.dtypes

dtype('int64')

In [6]:
#create heatmap layer 
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#create dataframe with only the columns we need (drop date column)
va_data_df = va_city_data_df.loc[:,["City Name","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country"]]
va_data_df.head()

#narrow down criteria 
narrow_va_data_df = va_data_df.loc[(va_data_df["Max Temp"]>72) & (va_data_df["Max Temp"]<78) &
                                   (va_data_df["Humidity"]<80) & (va_data_df["Cloudiness"]<25) &
                                   (va_data_df["Wind Speed"]<15)]

narrow_va_data_df                                    

#drop any rows with null values 
cleaned_va_data = narrow_va_data_df.dropna(how="any")
cleaned_va_data

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
24,Ţahţā,26.77,31.50,73.40,35,0,6.93,EG
68,Sur,22.57,59.53,76.41,58,0,9.71,OM
83,Isangel,-19.55,169.27,77.72,70,4,11.97,VU
252,De Aar,-30.65,24.01,77.18,18,0,6.29,ZA
344,Agadez,19.75,10.25,73.08,19,0,7.56,NE
403,Piaçabuçu,-10.41,-36.43,77.50,76,8,10.51,BR
502,Upington,-28.45,21.26,77.00,18,1,2.24,ZA
513,Salalah,17.02,54.09,77.00,36,0,3.36,OM
565,Bilma,18.69,12.92,74.26,18,0,4.72,NE


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#create a new dataframe for hotel info
hotel_df = cleaned_va_data

#add hotel name column 
hotel_df["Hotel Name"] = ""
hotel_df

#on the types, use lodging to find hotels 

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
24,Ţahţā,26.77,31.50,73.40,35,0,6.93,EG,
68,Sur,22.57,59.53,76.41,58,0,9.71,OM,
83,Isangel,-19.55,169.27,77.72,70,4,11.97,VU,
252,De Aar,-30.65,24.01,77.18,18,0,6.29,ZA,
344,Agadez,19.75,10.25,73.08,19,0,7.56,NE,
403,Piaçabuçu,-10.41,-36.43,77.50,76,8,10.51,BR,
502,Upington,-28.45,21.26,77.00,18,1,2.24,ZA,
513,Salalah,17.02,54.09,77.00,36,0,3.36,OM,
565,Bilma,18.69,12.92,74.26,18,0,4.72,NE,


In [36]:
#store "Lat" and "Lng" into locations
#locations = hotel_df[["Lat", "Lng"]].astype(float)
hotels = "lodging"
radius = 5000
#print(locations)

#set parameters and define base url

params = {
    #"location": locations, 
    "hotels": hotels,
    "radius": radius,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # add keyword to params dict
#     params["location_lat"] = row["Lat"].astype(float)
#     params["location_lng"] = row["Lng"].astype(float)
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #run request
    
    data = requests.get(base_url, params=params).json()
    #print(data)

   
   
    
    # extract results
    #results = response['results']
    
    try: 
        hotel_name = data["results"][0]["name"]
        print(f"Retrieving Results for Index {index}: {hotel_name}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name

    except (KeyError, IndexError): 
        print("Cannot find hotel for", row["City Name"])
        
    print("------------------")    
        
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

Retrieving Results for Index 24: Tahta.
------------------
Retrieving Results for Index 68: Sur.
------------------
Retrieving Results for Index 83: Isangel.
------------------
Retrieving Results for Index 252: De Aar.
------------------
Cannot find hotel for Agadez
------------------
Retrieving Results for Index 403: Piaçabuçu.
------------------
Retrieving Results for Index 502: Upington.
------------------
Retrieving Results for Index 513: Salalah.
------------------
Retrieving Results for Index 565: Bilma.
------------------


In [37]:
hotel_df

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
24,Ţahţā,26.77,31.50,73.40,35,0,6.93,EG,Tahta
68,Sur,22.57,59.53,76.41,58,0,9.71,OM,Sur
83,Isangel,-19.55,169.27,77.72,70,4,11.97,VU,Isangel
252,De Aar,-30.65,24.01,77.18,18,0,6.29,ZA,De Aar
344,Agadez,19.75,10.25,73.08,19,0,7.56,NE,
403,Piaçabuçu,-10.41,-36.43,77.50,76,8,10.51,BR,Piaçabuçu
502,Upington,-28.45,21.26,77.00,18,1,2.24,ZA,Upington
513,Salalah,17.02,54.09,77.00,36,0,3.36,OM,Salalah
565,Bilma,18.69,12.92,74.26,18,0,4.72,NE,Bilma


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
##add as a layer 

# Display figure
